In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
df=pd.read_csv('../Input/netflix_titles.csv')

In [3]:
netflix=df[['type','title','country','release_year','duration','listed_in','description']]
netflix=netflix[netflix.type=='Movie']
netflix=netflix[netflix.country=='United States']
netflix=netflix[netflix.release_year>1995]
netflix=netflix.head(1000)

In [4]:
apiKey = os.getenv("API_IH")
rapidapiKey = os.getenv("RAPIDAPI")

In [5]:
def getID(titulo, key):
    try:    
        url = "https://movie-database-imdb-alternative.p.rapidapi.com/"
        querystring = {"page":"1","r":"json","s":titulo}
        headers = {'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",'x-rapidapi-key': key}
        response = requests.request("GET", url, headers=headers, params=querystring)
        response=(response.text)
        response=((response.replace(' ','').replace('"imdbID":'," ").replace(',"Type":',' ').split())[1]).replace('\"',"")
    except:pass
    return(response)

In [6]:
netflix['id']=[getID(x,rapidapiKey) for x in netflix['title']]

In [7]:
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Movie not found!"}')]
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Too many results."}')]

In [8]:
net=netflix

In [9]:
def getFromTMDB(id_tmdb,i,key):
    try:
        res = (requests.get(f"https://api.themoviedb.org/3/movie/{id_tmdb}?api_key={key}&language=en-US")).json()[f'{i}']
    except:
        res = 'NaN'
    return res

In [10]:
for i in ['popularity','vote_count','vote_average']:
    net[f'{i}'] = [getFromTMDB(x,i,apiKey) for x in net['id']]

In [11]:
net = net[~(netflix.listed_in == 'Nan')]

In [22]:
net=net[['title','country','release_year','duration','listed_in','description','popularity','vote_count','vote_average']]

In [33]:
net.head()

,Title,Country,Year,Duration,Genre,Description,Popularity,Vote Count,Rating
4,#realityhigh,United States,2017,99 min,Comedies,When nerdy high schooler Dani finally attracts...,20.758,870,6.4
20,Manhattan Romance,United States,2014,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...,7.044,24,4.8
23,Stonehearst Asylum,United States,2014,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",26.797,880,6.7
24,The Runner,United States,2015,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,87.885,12504,7.1
25,6 Years,United States,2015,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...,15.477,624,5.4


In [30]:
net.rename(columns={'title':'Title','country':'Country','release_year':'Year','duration':'Duration','listed_in':'Genre','description':'Description','popularity':'Popularity','vote_count':'Vote Count','vote_average':'Rating'},inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [32]:
net.to_csv("../Output/NetflixFull.csv",index=False)

In [34]:
net.shape[0]

929

In [63]:
dev=['']
type(dev)

list

In [64]:
for i in ['Title','Country','Year','Duration','Genre','Description','Popularity','Rating']:
    x= net[i][net.Title=='American History X'].to_string(index=False)
    x=f'{i}: {x}'
    dev.append(x)
for i in range(1,9):
    print(dev[i])

    


Title:  American History X
Country:  United States
Year:  1998
Duration:  119 min
Genre:  Dramas
Description:  A neo-Nazi gets sent to prison for murder and ...
Popularity:  27.382
Rating:  8.4
